In [101]:
from os import listdir
import pandas as pd
import numpy as np

In [102]:
TEXTS_PATH1 = "data2/datasets1/train-articles"
LABELS_PATH1 = "data2/datasets1/train-labels-task2-technique-classification"

TEXTS_PATH2 = "data2/datasets2/train-articles"
LABELS_PATH2 = "data2/datasets2/train-labels-task2-technique-classification"

In [103]:
from os import listdir

TEXTS_NAMES1 = listdir(TEXTS_PATH1)
TEXTS_NAMES2 = listdir(TEXTS_PATH2)
PART_TEXT_NAMES = np.concatenate([TEXTS_NAMES1, TEXTS_NAMES2])

LABELS_NAMES = []
TEXTS_NAMES = []
for i, name in enumerate(PART_TEXT_NAMES): 
    if i < len(TEXTS_NAMES1):
        TEXTS_NAMES += [TEXTS_PATH1 + "/" + name]
        LABELS_NAMES += [LABELS_PATH1 + "/" + name[:-3] + "task2-TC.labels"]
    else:
        TEXTS_NAMES += [TEXTS_PATH2 + "/" + name]
        LABELS_NAMES += [LABELS_PATH2 + "/" + name[:-3] + "task2-TC.labels"]

In [104]:
def read_labels_file(label_path):
    f = open(label_path, "r")
    labels = []
    for string in f:
        num, technique, start, end = string.split("\t")
        labels += [[int(start), int(end[:-1]), technique]]
    labels.sort()
    return labels

def read_text_file(text_path):
    f = open(text_path, "r")
    return f.read()

In [105]:
def separate_text(text, labels):
    sep_data = []
    positions = [0]
    texts = text.split("\n")
    ind = 0
    for i in range(len(texts)):
        ind += len(texts[i]) + 1
        positions += [ind]
    c_pos = 0
    for label in labels:
        while (positions[c_pos] <= label[0] < positions[c_pos + 1]) == 0:
            c_pos += 1
            if c_pos + 1 == len(positions):
                return sep_data
        left_edge = positions[c_pos]
        sep_data += [[texts[c_pos], 
                      label[0] - left_edge,
                      label[1] - left_edge,
                      label[2]]]
    return sep_data

In [106]:
def add_json_type(d):
    el = [{
            "start": d[1],
            "end": d[2],
            "technique": d[3]
         }]
    return el

In [107]:
labels = []
texts = []
data = []
for i in range(len(TEXTS_NAMES)):
    text_path = TEXTS_NAMES[i]
    label_path = LABELS_NAMES[i]
    labels = read_labels_file(label_path)
    text = read_text_file(text_path)
    sep_data = separate_text(text, labels)
    if len(sep_data) == 0:
        continue
    c_text = sep_data[0][0]
    label = []
    for d in sep_data:
        if d[0] == c_text:
            label += add_json_type(d)
        else:
            if len(label) != 0:
                data += [[c_text, label]]
            c_text = d[0]
            label = add_json_type(d)
    data += [[c_text, label]]

In [108]:
data_pd = pd.DataFrame(data, columns=["text", "labels"])

In [109]:
labels = [
    "Reductio ad hitlerum",
    "Whataboutism",
    "Presenting Irrelevant Data (Red Herring)",
    "Doubt",
    "Slogans",
    "Appeal to fear/prejudice",
    "Obfuscation, Intentional vagueness, Confusion",
    "Misrepresentation of Someone's Position (Straw Man)",
    "Glittering generalities (Virtue)",
    "Appeal to authority",
    "Repetition",
    "Bandwagon",
    "Causal Oversimplification",
    "Name calling/Labeling",
    "Thought-terminating cliché",
    "Flag-waving",
    "Exaggeration/Minimisation",
    "Smears",
    "Loaded Language",
    "Black-and-white Fallacy/Dictatorship"
]

In [110]:
new_corpus_labels = [
    "Loaded_Language",
    "Name_Calling,Labeling",
    "Repetition",
    "Exaggeration,Minimisation",
    "Doubt",
    "Appeal_to_fear-prejudice",
    "Flag-Waving",
    "Causal_Oversimplification",
    "Slogans",
    "Appeal_to_Authority",
    "Black-and-White_Fallacy",
    "Thought-terminating_Cliches",
    "Whataboutism",
    "Reductio_ad_Hitlerum",
    "Red_Herring",
    "Bandwagon",
    "Obfuscation,Intentional_Vagueness,Confusion",
    "Straw_Men"
]

In [111]:
new_corpus_labels.sort()
labels.sort()
labels.remove("Glittering generalities (Virtue)")
labels.remove("Misrepresentation of Someone's Position (Straw Man)")

In [112]:
print(len(labels))
print(len(new_corpus_labels))

18
18


In [113]:
d_change_labels = dict()

for i in range(len(labels)):
    d_change_labels[new_corpus_labels[i]] = labels[i]

d_change_labels["Straw_Men"] = "Misrepresentation of Someone's Position (Straw Man)"

In [114]:
d_change_labels

{'Appeal_to_Authority': 'Appeal to authority',
 'Appeal_to_fear-prejudice': 'Appeal to fear/prejudice',
 'Bandwagon': 'Bandwagon',
 'Black-and-White_Fallacy': 'Black-and-white Fallacy/Dictatorship',
 'Causal_Oversimplification': 'Causal Oversimplification',
 'Doubt': 'Doubt',
 'Exaggeration,Minimisation': 'Exaggeration/Minimisation',
 'Flag-Waving': 'Flag-waving',
 'Loaded_Language': 'Loaded Language',
 'Name_Calling,Labeling': 'Name calling/Labeling',
 'Obfuscation,Intentional_Vagueness,Confusion': 'Obfuscation, Intentional vagueness, Confusion',
 'Red_Herring': 'Presenting Irrelevant Data (Red Herring)',
 'Reductio_ad_Hitlerum': 'Reductio ad hitlerum',
 'Repetition': 'Repetition',
 'Slogans': 'Slogans',
 'Straw_Men': "Misrepresentation of Someone's Position (Straw Man)",
 'Thought-terminating_Cliches': 'Thought-terminating cliché',
 'Whataboutism': 'Whataboutism'}

In [115]:
def correct_technique(old_t, name, data, i):
    old_t["technique"] = name
    data["labels"][i].append(old_t)

In [116]:
wrong_name1 = ["Bandwagon", "Reductio_ad_Hitlerum"]
wrong_name2 = ["Whataboutism", "Straw_Men", "Red_Herring"]

def change_technique_name(data, d_change_labels):
    for i in range(data.shape[0]):
        N = len(data["labels"][i])
        j = 0
        while j < N:
            old_name = data["labels"][i][j]["technique"]
            if "Bandwagon,Reductio_ad_hitlerum" == old_name:
                old_t = data["labels"][i].pop(j)
                for name in wrong_name1:
                    correct_technique(old_t.copy(), name, data, i)
                continue
            if "Whataboutism,Straw_Men,Red_Herring" == old_name:
                old_t = data["labels"][i].pop(j)
                for name in wrong_name2:
                    correct_technique(old_t.copy(), name, data, i)
                continue
            new_name = d_change_labels[data["labels"][i][j]["technique"]]
            data["labels"][i][j]["technique"] = new_name
            j += 1

In [117]:
change_technique_name(data_pd, d_change_labels)

In [118]:
data_pd.to_csv("data2/datasets1/new_data2.csv", index=False)